In [1]:
import sqlite3
import sys
import csv
from pprint import pprint
from collections import defaultdict

# San Jose
#### From Open Street Map 

## create and connect database

In [2]:
sqlite_file = "openstreetmap.db"
conn = sqlite3.connect(sqlite_file)

## insert csv file into database

In [3]:
cur = conn.cursor()
with open('nodes_tags.csv','rb') as fin1:
    dr1 = csv.DictReader(fin1) 
    to_nodes_tags = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"),i['value'].decode("utf-8"),
                      i['type'].decode("utf-8")) 
                     for i in dr1]
    
with open('nodes.csv','rb') as fin2:
    dr2 = csv.DictReader(fin2) 
    to_nodes = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"),i['uid'].decode("utf-8"),
                 i['version'].decode("utf-8"),  i['lat'].decode("utf-8"), i['lon'].decode("utf-8"), 
                 i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) 
                     for i in dr2]
    
with open('ways.csv','rb') as fin3:
    dr3 = csv.DictReader(fin3) 
    to_ways = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"),i['uid'].decode("utf-8"),
                i['version'].decode("utf-8"),  i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) 
                     for i in dr3]
    
with open('ways_tags.csv','rb') as fin4:
    dr4 = csv.DictReader(fin4) 
    to_ways_tags = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"),i['value'].decode("utf-8"), i['type'].decode("utf-8")) 
                     for i in dr4]
    
with open('ways_nodes.csv','rb') as fin5:
    dr5 = csv.DictReader(fin5) 
    to_ways_nodes = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) 
                     for i in dr5]
    
cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_nodes_tags)
cur.executemany("INSERT INTO nodes(id, user, uid, version,lat,lon, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_nodes)
cur.executemany("INSERT INTO way(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_ways)
cur.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_ways_tags)
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_ways_nodes)

conn.commit()
conn.close()


## count entries for each file 

In [5]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''SELECT COUNT(*) FROM nodes
            ''')
output1=cur.fetchall()


cur.execute('''SELECT COUNT(*) FROM nodes_tags
            ''')
output2=cur.fetchall()

cur.execute('''SELECT COUNT(*) FROM way
            ''')
output3=cur.fetchall()

cur.execute('''SELECT COUNT(*) FROM ways_tags
            ''')
output4=cur.fetchall()

cur.execute('''SELECT COUNT(*) FROM ways_nodes
            ''')
output5=cur.fetchall()

print "nodes's data: ", output1[0][0]
print "nodes_tags's data: " , output2[0][0]
print "way's data: " , output3[0][0]
print "ways_tags's data: " , output4[0][0]
print "ways_nodes's data: " , output5[0][0]
conn.commit()
conn.close()

nodes's data:  1276635
nodes_tags's data:  66320
way's data:  169712
ways_tags's data:  615127
ways_nodes's data:  1491733


## Unique users and Top Ten contributing users

In [6]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''SELECT COUNT(DISTINCT(M.uid))
                FROM (  SELECT uid FROM nodes
                        UNION ALL
                        SELECT uid FROM way) M
            ''')
output=cur.fetchall()
print "Total unique users: ", output[0][0],"\n"
cur.execute('''SELECT uid, user, COUNT(uid) AS nums
            FROM nodes
            GROUP BY uid
            ORDER BY nums DESC
            limit 10
            ''')
output=cur.fetchall()
print "Top Ten contributing users:\n"
for row in output:
    print "id: ",row[0] , ", User Name: ", row[1], ", Edited times: ", row[2]
conn.commit()
conn.close()

Total unique users:  1186 

Top Ten contributing users:

id:  55774 , User Name:  nmixter , Edited times:  252497
id:  201724 , User Name:  mk408 , Edited times:  143696
id:  74705 , User Name:  Bike Mapper , Edited times:  71701
id:  2512300 , User Name:  samely , Edited times:  68396
id:  2226712 , User Name:  dannykath , Edited times:  66277
id:  2219338 , User Name:  RichRico , Edited times:  63673
id:  2748195 , User Name:  karitotp , Edited times:  51835
id:  318696 , User Name:  n76 , Edited times:  31278
id:  595221 , User Name:  matthieun , Edited times:  28199
id:  33757 , User Name:  Minh Nguyen , Edited times:  27496


## Top 5 version used

In [58]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''SELECT version, COUNT(version) AS nums
            FROM nodes
            GROUP BY version
            ORDER BY nums DESC
            limit 5
            ''')
output=cur.fetchall()
for row in output:
    print "version " , row[0], ", number :", row[1]
conn.commit()
conn.close()

version  1 , number : 991878
version  2 , number : 171662
version  3 , number : 46053
version  4 , number : 27034
version  5 , number : 16727


In [59]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''SELECT version, COUNT(version) AS nums
            FROM nodes
            WHERE user="nmixter"
            GROUP BY version
            ORDER BY nums DESC
            limit 5
            ''')
output=cur.fetchall()
for row in output:
    print "version " , row[0], ", number :", row[1]
    

cur.execute('''SELECT version, COUNT(version) AS nums
            FROM nodes
            WHERE (user="nmixter" or user = "mk408")
            GROUP BY version
            ORDER BY nums DESC
            limit 5
            ''')
output=cur.fetchall()
for row in output:
    print "version " , row[0], ", number :", row[1]
conn.commit()
conn.close()

version  1 , number : 244311
version  2 , number : 5578
version  3 , number : 992
version  4 , number : 566
version  5 , number : 399
version  1 , number : 377882
version  2 , number : 8908
version  4 , number : 3009
version  3 , number : 2228
version  5 , number : 2087


# Contributor statistics
## overall
+ Top user "nmixter" contributed 19.78% of the data
+ Top five users contributed 47.20% of the data
+ Top ten users contributed 63.06% of the data

#### we also want to know the contribution base on the version
#### we find the first version that is most used version.

## base on version
+ Top user "nmixter" contribution is 24.63% in the first version
+ Top two users contribution is 38.01% in the first version


## postcode check
##### we try to find out users input non digits into postcode 

In [8]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''
            SELECT U.user
            FROM(
            SELECT way.user
            FROM ways_tags, way
            WHERE way.uid=ways_tags.id
            and ways_tags.key= "postcode"
            and ways_tags.value=""
            UNION ALL
            SELECT nodes.user
            FROM nodes_tags, nodes
            WHERE nodes.id=nodes_tags.id
            and nodes_tags.key="postcode"
            and nodes_tags.value='') U

            ''')
output=cur.fetchall()
print output



conn.commit()
conn.close()

[(u'n76_cupertino_import',)]


## Top 5 City

In [16]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute(''' UPDATE ways_tags
                SET value='San Jose'
                WHERE type = "addr"
                and key="city" 
                and (value = "San jose" or value = "san jose" or value = "San Jos\xe9" )

            ''')
output=cur.fetchall()
cur.execute('''SELECT ways_tags.value, COUNT(ways_tags.value) AS count
                FROM ways_tags
                WHERE ways_tags.type = "addr"
                and ways_tags.key="city"
                GROUP BY ways_tags.value
                ORDER BY count DESC
                LIMIT 5
            ''')
output=cur.fetchall()
print "Top 5 city \n"
for row in output:
    print "City's name: " ,row[0], ";  count: ", row[1]

conn.commit()
conn.close()

Top 5 city 

City's name:  Sunnyvale ;  count:  663
City's name:  San Jose ;  count:  475
City's name:  Morgan Hill ;  count:  340
City's name:  Santa Clara ;  count:  214
City's name:  Saratoga ;  count:  183


## Top 10 amenity in San Jose

In [17]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
cur.execute('''
          SELECT value, COUNT(*) as num
            FROM nodes_tags
            WHERE key='amenity'
            GROUP BY value
            ORDER BY num DESC
            LIMIT 10; 
            ''')

    
cur.execute('''
          SELECT value, COUNT(*) as num
            FROM nodes_tags
            WHERE key='amenity'
            GROUP BY value
            ORDER BY num DESC
            LIMIT 10; 
            ''')

                        

output=cur.fetchall()
print "Top 5 amenity \n"
for row in output:
    print "amenity: " ,row[0], ";  count: ", row[1]


Top 5 amenity 

amenity:  restaurant ;  count:  740
amenity:  fast_food ;  count:  354
amenity:  cafe ;  count:  220
amenity:  place_of_worship ;  count:  189
amenity:  bench ;  count:  170
amenity:  bicycle_parking ;  count:  166
amenity:  school ;  count:  157
amenity:  toilets ;  count:  142
amenity:  fuel ;  count:  124
amenity:  bank ;  count:  114


In [62]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

    
cur.execute('''
            SELECT nodes_tags.value, COUNT(*) as num
            FROM nodes_tags 
            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='fast_food') i
            ON nodes_tags.id=i.id
            WHERE nodes_tags.key='name'
            GROUP BY nodes_tags.value
            ORDER BY num DESC
            LIMIT 10;
            ''')

                        

output=cur.fetchall()
for row in output:
    print row[1], row[0] 

33 Subway
12 Taco Bell
10 McDonald's
8 Burger King
8 Panda Express
8 Togo's
7 Jamba Juice
7 KFC
6 Jack in the Box
5 Baskin-Robbins


### Fast food in San Jose
>we can see around 10% of Fast Food franchise is Subway. this is not a suprise because Subway is the largest franchise.

######  San Jose has many good restaurant , we want to know what type of food

# restaurant group by cuisine

In [47]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

    
cur.execute('''
            SELECT nodes_tags.value, COUNT(*) as num
            FROM nodes_tags 
            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
            ON nodes_tags.id=i.id
            WHERE nodes_tags.key='cuisine'
            GROUP BY nodes_tags.value
            ORDER BY num DESC
            LIMIT 10;
            ''')

                        

output=cur.fetchall()
for row in output:
    print row[1], " are", row[0], " cuisine"

62  are chinese  cuisine
49  are vietnamese  cuisine
48  are mexican  cuisine
47  are pizza  cuisine
38  are japanese  cuisine
30  are indian  cuisine
27  are italian  cuisine
24  are thai  cuisine
22  are american  cuisine
19  are sushi  cuisine


### Restaurant
+ Top 10 cuisine is only 48% of the restaurant.
+ we can say that San Jose very diverse in food.

+ we also want to know the name of those restaurant.

# restaurant name group by cuisine

In [48]:
restaurant=defaultdict(set)
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

    
cur.execute('''
            SELECT  name.value, cuisine.value
            
            FROM (SELECT id, value
                    FROM nodes_tags
                    WHERE id IN (SELECT id
                                FROM nodes_tags
                                WHERE value = "restaurant")
                    AND key = "cuisine") cuisine
            JOIN (SELECT id, value
                    FROM nodes_tags
                    WHERE id IN (SELECT id
                                FROM nodes_tags
                                WHERE value = "restaurant")
                    AND key = "name") name
            ON name.id = cuisine.id


            ''')

                        

output=cur.fetchall()

for row in output:
    restaurant[row[1]].add(row[0])
pprint (dict(restaurant))
conn.commit()
conn.close()

{u'Mediterranean,_wine_tasting, beer tasting, live music': set([u'Santorini Wine and Beer Garden']),
 u'Persian': set([u'House of Kabobs']),
 u'Traditional American': set([u"Vahl's Restaurant & Cocktail"]),
 u'Vietnamese': set([u'Pho Spot', u'Pho Wagon']),
 u'afgan': set([u'Kabul']),
 u'american': set([u"Applebee's",
                   u'Bill of Fare',
                   u'Billy Berk\u2019s',
                   u"Birk's Restaurant",
                   u"Chili's Grill & Bar",
                   u'Country Inn Cafe II',
                   u'Denny\u2019s',
                   u'Elephant Bar',
                   u"Eric's Delli Cafe",
                   u'Fahrenheit',
                   u"Hobee's",
                   u'IHOP',
                   u"J.J. Magoo's Pizza",
                   u"John's Cafe",
                   u'Longhorn restaurant',
                   u"Marie Callender's",
                   u"Mimi's Cafe",
                   u'Stacks',
                   u"T.G.I. Friday's",
      

## Conclusion
> After looking at the result, we can see half of the data set is getting input by only few users, the good thing about that is the format of the data is not too different, and users who can insert large amount of data is people who understand the open street map really well speically on the version one. For new users want to contribute to the data set, they should find those contrubute the most and learn from them. 

#### suggestion
>open street map should prove a guildline for new users, teach the way to input data in the same way,such as capital letter, abbreviation. Gives example about how to break down the data into different field. Some users put all the address in one field and some break it down to street, city , state. Open street map should give a guildline about how to insert the data, so it will be easier for people to use the data set. 

>we also notice that many of the tag type only have one or two data. If the open street map should limit or only let people choose from a list of item, people do not need to create different tags key or tags type. This will make people want to analysis the data easier and do not have to many confusion about different type that users create on there own.